# ETL Proyect

### Instructions

* Create a `Vehicles_db` database in pgAdmin 4 then create the following  tables within:

  * A `vehicles` table that contains the data from CSV files(kaggle).

  * A `vehicles_craig` data that we get from scrapping web from Craighlist.




In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
from bs4 import BeautifulSoup
import requests

### Read  CSV

In [3]:
vehicles_file = os.path.join("..", "Resources", "vehicles.csv")
vehicles_df = pd.read_csv(vehicles_file)
vehicles_df.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,...,vin,drive,size,type,paint_color,description,county,state,lat,long
0,7034441763,salt lake city,17899,2012,volkswagen,golf r,excellent,4 cylinders,gas,63500,...,WVWPF7AJ6CW316713,4wd,compact,hatchback,black,PRICE REDUCED! -Garage kept -Low Miles (63K)...,NaN,ut,40.7372,-111.858
1,7034440610,salt lake city,0,2016,ford,f-150,excellent,NaN,gas,10,...,1FTMF1EP3GKF13544,4wd,NaN,NaN,NaN,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
2,7034440588,salt lake city,46463,2015,gmc,sierra 1500,excellent,NaN,gas,7554,...,3GTU2WEC6FG228025,4wd,NaN,NaN,white,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
3,7034440546,salt lake city,0,2016,ford,f-150,excellent,NaN,gas,10,...,1FTEX1EF6GKD25447,4wd,NaN,NaN,NaN,Drive it home today. Call (Or Text) us now !!C...,NaN,ut,40.5881,-111.884
4,7034406932,salt lake city,49999,2018,ford,f-450,NaN,NaN,diesel,70150,...,1FT8W4DT8GEA90427,4wd,NaN,pickup,white,2018 Ford F-350 F350 F 350 SD Lariat Crew Cab ...,NaN,ut,40.3744,-104.694


In [4]:
### Create a filtered dataframe from specific columns

vehicle_cols = ["id", "region", "year","manufacturer", "model", "paint_color", "state"]
vehicle_transformed= vehicles_df[vehicle_cols].copy()


# Clean the data by dropping duplicates and setting the index

vehicle_transformed.drop_duplicates("id", inplace=True)
vehicle_transformed.set_index("id", inplace=True)

vehicle_transformed.head()

,region,year,manufacturer,model,paint_color,state
id,,,,,,
7034441763,salt lake city,2012,volkswagen,golf r,black,ut
7034440610,salt lake city,2016,ford,f-150,NaN,ut
7034440588,salt lake city,2015,gmc,sierra 1500,white,ut
7034440546,salt lake city,2016,ford,f-150,NaN,ut
7034406932,salt lake city,2018,ford,f-450,white,ut


In [22]:
manuf = vehicle_transformed.groupby('manufacturer')
total_manuf = manuf['region'].count()
total_manuf_df = pd.DataFrame(total_manuf)
total_manuf_df  


,region
manufacturer,
tire rack,18
acura,4169
alfa-romeo,109
aston-martin,22
audi,5586
bmw,13468
buick,6357
cadillac,7304
chevrolet,73661


### URL of page to be scraped

In [23]:
#Craighlist page
url = 'https://houston.craigslist.org/d/cars-trucks/search/cta?s=240'

### Retrieve page with the requests module

In [7]:
response = requests.get(url)

### Create BeautifulSoup object

In [8]:
soup = BeautifulSoup(response.text, 'html.parser')

### Check the results

In [24]:
print(soup.prettify())

﻿
<!DOCTYPE html>
<html class="no-js">
 <head>
  <title>
   houston cars &amp; trucks  - craigslist
  </title>
  <meta content="houston cars &amp; trucks  - craigslist" name="description"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible">
   <link href="https://houston.craigslist.org/search/cta?s=240" rel="canonical"/>
   <link href="https://houston.craigslist.org/search/cta?format=rss&amp;s=240" rel="alternate" title="RSS feed for craigslist | houston cars &amp; trucks  - craigslist" type="application/rss+xml"/>
   <link href="https://houston.craigslist.org/search/cta?s=120" rel="prev"/>
   <link href="https://houston.craigslist.org/search/cta?s=360" rel="next"/>
   <meta content="width=device-width,initial-scale=1" name="viewport"/>
   <link href="//www.craigslist.org/styles/cl.css?v=f45c4074e9853c0ceb280398a8131359" media="all" rel="stylesheet" type="text/css"/>
   <link href="//www.craigslist.org/styles/search.css?v=bc035cbbc3978b0ec9df93944cdf349b" media="all" rel="styleshe

### Results in a List

In [25]:
results = soup.find_all('li', class_="result-row")
print(len(results))

120


In [26]:
print(results[0].prettify())

<li class="result-row" data-pid="7054325181">
 <a class="result-image gallery" data-ids="1:00505_aipgiZABB5v,1:00z0z_laLGnfqCvon,1:00V0V_9HSPF2AZepG,1:00101_abT80UbDtY0,1:00B0B_6U5kYuXL6Ll,1:00202_blhs7Tl9IS0,1:00909_i8w2N9CB76L,1:00Y0Y_8XcxQ4pyBSi,1:00707_8gWXhToo9rz,1:00q0q_hJAz10FuGxj,1:00A0A_vQi563bAfG,1:00101_gkNTnKCFeU4,1:00u0u_6Mn9dls0Hys,1:00m0m_3HSLpsHiTqv,1:00n0n_bfaoQGkoLPE,1:00F0F_lFnzThjbdCK,1:00t0t_anieZf8vHD0,1:00303_dsEOnvrmOtr,1:00V0V_4ouAHYD4oNB,1:01414_2hZ7F6uE4I3,1:00l0l_eC8vTddf4L0,1:00X0X_T3RqqgAAsr" href="https://houston.craigslist.org/cto/d/north-houston-2004-dodge-ram-pickup/7054325181.html">
  <span class="result-price">
   $11980
  </span>
 </a>
 <p class="result-info">
  <span class="icon icon-star" role="button">
   <span class="screen-reader-text">
    favorite this post
   </span>
  </span>
  <time class="result-date" datetime="2020-01-11 16:01" title="Sat 11 Jan 04:01:01 PM">
   Jan 11
  </time>
  <a class="result-title hdrlnk" data-id="7054325181" href=

### Loop the results

In [27]:
title_ls = []
price_ls = []
hood_ls = []
link_ls = []


for result in results:
    
    # Error handling
    try:
        
        # Identify and return title of listing
        title = result.find('a', class_="result-title").text
        
        # Identify and return price of listing
        price = result.find('span', class_="result-price").text
        
        hood = result.find('span', class_= "result-hood").text        
      
    
        # Identify and return link to listing
        link = result.a['href']

        # Print results only if title, price, and link are available
        if (title and price and hood and link):
            print('-------------')
            print(title)
            title_ls.append(title)
            print(price)
            price_ls.append(price)
            print(hood)
            hood_ls.append(hood)
            print(link)
            link_ls.append(link)
            
    except AttributeError as e:
        print(e)

'NoneType' object has no attribute 'text'
-------------
2017 FORD FOCUS SE
$6900
 (richmond)
https://houston.craigslist.org/cto/d/richmond-2017-ford-focus-se/7054324938.html
-------------
2013 *Jeep* *Wrangler* *Rubicon* Commando Green
$27997
 (Emmons Motor Company)
https://houston.craigslist.org/ctd/d/south-houston-2013-jeep-wrangler/7054324640.html
-------------
2005 BLUE BIRD ALL AMERICAN  CUMMINS 8.3 250HP
$6500
 (N Houston)
https://houston.craigslist.org/cto/d/houston-2005-blue-bird-all-american/7054323708.html
'NoneType' object has no attribute 'text'
-------------
Hyundai Sonata
$3000
 (West Houston)
https://houston.craigslist.org/cto/d/houston-hyundai-sonata/7054322916.html
-------------
2015 HONDA ACCORD SPORT  100% IN-HOUSE FINANCING - BUY HERE PAY HERE
$1495
 (DOWN ** WE SPECIALZED WITH IN HOUSE FINANCING)
https://houston.craigslist.org/ctd/d/2015-honda-accord-sport-100-in-house/7054322402.html
-------------
2013 Ford Explorer
$7300
 (Southwest Houston)
https://houston.craig

In [14]:
# Merge list
ziplist = list(zip(title_ls, price_ls, hood_ls, link_ls))

In [15]:
# Create Date frame
vehicles_craig = pd.DataFrame(ziplist, columns = ['Title', 'Price','Neighborhood', 'Link'])

vehicles_craig.head()

,Title,Price,Neighborhood,Link
0,2017 FORD FOCUS SE,$6900,(richmond),https://houston.craigslist.org/cto/d/richmond-...
1,2013 *Jeep* *Wrangler* *Rubicon* Commando Green,$27997,(Emmons Motor Company),https://houston.craigslist.org/ctd/d/south-hou...
2,2005 BLUE BIRD ALL AMERICAN CUMMINS 8.3 250HP,$6500,(N Houston),https://houston.craigslist.org/cto/d/houston-2...
3,Hyundai Sonata,$3000,(West Houston),https://houston.craigslist.org/cto/d/houston-h...
4,2015 HONDA ACCORD SPORT 100% IN-HOUSE FINANCI...,$1495,(DOWN ** WE SPECIALZED WITH IN HOUSE FINANCING),https://houston.craigslist.org/ctd/d/2015-hond...


### Create database connection

In [16]:
pg_user = 'postgres'
pg_password = 'luar6902'
db_name = 'Vehicles_db'

connection_string = f"{pg_user}:{pg_password}@localhost:5432/{db_name}"
engine = create_engine(f'postgresql://{connection_string}')

### Confirm tables

In [17]:
engine.table_names()

['vehicles', 'vehicles_craig']

### Load DataFrames into database

In [42]:
vehicle_transformed.to_sql(name='vehicles', con=engine, if_exists='append', index=True)

In [86]:
vehicles_craig.to_sql(name = 'vehicles_craig', con = engine, if_exists='append', index=True)

In [18]:
pd.read_sql_query('select * from vehicles', con=engine).head()

,id,region,year,manufacturer,model,paint_color,state
0,7034441763,salt lake city,2012.0,volkswagen,golf r,black,ut
1,7034440610,salt lake city,2016.0,ford,f-150,None,ut
2,7034440588,salt lake city,2015.0,gmc,sierra 1500,white,ut
3,7034440546,salt lake city,2016.0,ford,f-150,None,ut
4,7034406932,salt lake city,2018.0,ford,f-450,white,ut


In [19]:
pd.read_sql_query('select * from vehicles_craig', con=engine).head()

,index,Title,Price,Neighborhood,Link
0,0,2006 Toyota 4Runner SR5 4WD 4dr SUV 4x4,$7285,(Houston),https://houston.craigslist.org/ctd/d/houston-2...
1,1,2007 f150 king ranch,$6899,(Spring),https://houston.craigslist.org/cto/d/spring-20...
2,2,TOYOTA TUNDRA SR5--2015--REVCAM NAVI CREWMAX C...,$18900,(A/F WE FINANCE)(CALL DEXTER 3462298640),https://houston.craigslist.org/ctd/d/houston-t...
3,3,CHEVROLET SILVERADO LT Texas EDT--2014--Naviga...,$16900,(A/F WE FINANCE)(CALL DEXTER 3462298640),https://houston.craigslist.org/ctd/d/houston-c...
4,4,2006 Dodge Ram,$1300,(HOUSTON),https://houston.craigslist.org/cto/d/houston-2...
